#Phase 3: Query Retrieval System Implementation
This implementation utilizes Llama 3 as a free alternative to OpenAI. While Llama 3 can be run locally via Ollama, the most efficient approach for Google Colab is using the Groq API. Groq is currently free, exceptionally fast, and prevents the Colab environment from exhausting its RAM by offloading the inference from the local machine. Feel free to change it to OpenAI or Ollama based on your requirements.

## Step 1: Setup LLM Integration and Neo4j Connection

In [ ]:
!pip install groq neo4j -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 10.9 MB/s eta 0:00:00


In [ ]:
# --- INITIALIZATION ---
# Replace with your actual Neo4j and Groq credentials
NEO4J_URI = os.getenv("NEO4J_URI", "ENTER_URI_HERE")
NEO4J_USER = os.getenv("NEO4J_USER", "ENTER_USER_HERE")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "ENTER_PASS_HERE")
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "ENTER_API_KEY_HERE")

In [ ]:
import os
from groq import Groq
from neo4j import GraphDatabase

# --- From PHASE 2: Connection Class ---
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.uri = uri
        self.user = user
        self.password = password
        self.driver = None

        try:
            from neo4j import GraphDatabase
            self.driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
            # Test connection
            with self.driver.session() as session:
                result = session.run("RETURN 'Neo4j Connection Successful' AS message")
                print(result.single()["message"])
        except Exception as e:
            print(f"Error connecting to Neo4j: {e}")
            # For Colab demo, we'll create a mock connection
            self.driver = None

    def close(self):
        self.driver.close()

    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return [record.data() for record in result]

# --- PHASE 3: Llama 3 Query Engine ---
class AviationKGQueryEngine:
    def __init__(self, neo4j_conn, groq_api_key):
        self.conn = neo4j_conn
        self.client = Groq(api_key=groq_api_key)
        self.model = "llama-3.3-70b-versatile"

        # We provide the schema context so Llama 3 writes valid Cypher for your graph
        self.schema_context = """
        Nodes:
        - Accident {event_id, event_date, location, severity}
        - Aircraft {make, model, registration_number}
        - Airline {airline_name}
        - Airport {airport_code, airport_name}

        Relationships:
        - (Aircraft)-[:INVOLVED_IN]->(Accident)
        - (Accident)-[:OCCURRED_AT]->(Airport)
        - (Aircraft)-[:OPERATED_BY]->(Airline)
        """

    def query(self, question):
        try:
            # 1. Generate Cypher using Llama 3
            prompt = f"""
                        Task: Write a Neo4j Cypher query to answer the user's question based on the provided schema.
                        Schema: {self.schema_context}
                        Question: {question}

                        Rules:
                        1. Only return the Cypher query. No explanations.
                        2. Use Case-Insensitive matching for strings.
                        3. Limit results to 10 unless specified otherwise.
                        """

            cypher_prompt = f"Write a Neo4j Cypher query for: '{question}'. Schema: {self.schema_context}. Return ONLY the query code."
            cypher_res = self.client.chat.completions.create(
                messages=[{"role": "user", "content": cypher_prompt}],
                model=self.model
            )
            cypher_query = cypher_res.choices[0].message.content.strip().replace('```cypher', '').replace('```', '')

            # 2. Run query in Neo4j
            graph_data = self.conn.execute_query(cypher_query)

            # 3. Summarize results using Llama 3
            summary_prompt = f"Question: {question}\nData: {graph_data}\nSummarize this aviation safety finding briefly."
            summary_res = self.client.chat.completions.create(
                messages=[{"role": "user", "content": summary_prompt}],
                model=self.model
            )
            summary = summary_res.choices[0].message.content

            return {
                "success": True,
                "summary": summary,
                "results": graph_data,
                "raw_cypher": cypher_query,
                "execution_metrics": {"time_seconds": 1.2}
            }
        except Exception as e:
            return {"success": False, "error": str(e)}

    def get_system_metrics(self):
        # Placeholder for your dashboard metrics
        return {"performance": {"total_queries_processed": 1, "success_rate": 1.0}}

# Now 'Neo4jConnection' is defined and can be instantiated
conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
query_engine = AviationKGQueryEngine(conn, GROQ_API_KEY)

Neo4j Connection Successful


Test query using the integrated query engine

In [ ]:
query_engine.query("Show all BOEING flights at KLAX")

{'success': True,
 'summary': 'The data consists of 10 aviation safety reports related to Boeing 737-800 flights at Los Angeles International Airport (KLAX). The reports involve passenger flights operated by United Airlines, with various event IDs and investigation types. The make of the aircraft is consistently listed as Boeing, and the model is 737-800. The data does not provide information on specific flight numbers, departure or arrival times, or the nature of the safety incidents. Overall, the reports appear to be related to accidents or incidents that occurred in 2021, with investigations completed by 2022.',
 'results': [{'a': {'purpose_of_flight': 'Passenger',
    'source_dataset': 'NTSB',
    'aircraft_category': 'Airplane',
    'registration_number': 'N12345',
    'airport_code': nan,
    'injury_severity': nan,
    'report_status': nan,
    'entity_id': 'aircraft_N12345',
    'airline_name': nan,
    'event_id': nan,
    'airport_name': nan,
    'event_date': nan,
    'confi

## Step 2: Serving it as Flask Micro Service

In [ ]:
!npm install -g localtunnel
!pip install Flask

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠸Requirement already satisfied: Flask in /usr/local/lib/python3.12/dist-packages (3.1.2)


In [ ]:
import threading
import sqlite3
from flask import Flask, request, jsonify, render_template_string
import math

class AviationQueryAPI:
    def __init__(self, query_engine):
        self.query_engine = query_engine
        self.app = Flask(__name__)

        @self.app.route('/')
        def home():
            # Paste your entire HTML string here
            return render_template_string('''
            <!DOCTYPE html>
            <html>
            <head>
                <title>Aviation Safety Query Dashboard</title>
                <style>
                    body {
                        font-family: Arial, sans-serif;
                        max-width: 1200px;
                        margin: 0 auto;
                        padding: 20px;
                        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                        min-height: 100vh;
                    }
                    .container {
                        background: white;
                        border-radius: 10px;
                        padding: 30px;
                        box-shadow: 0 10px 30px rgba(0,0,0,0.2);
                    }
                    h1 {
                        color: #333;
                        text-align: center;
                        margin-bottom: 30px;
                    }
                    .query-form {
                        display: flex;
                        gap: 10px;
                        margin-bottom: 20px;
                    }
                    input[type="text"] {
                        flex: 1;
                        padding: 12px;
                        border: 2px solid #ddd;
                        border-radius: 5px;
                        font-size: 16px;
                    }
                    button {
                        padding: 12px 24px;
                        background: #4CAF50;
                        color: white;
                        border: none;
                        border-radius: 5px;
                        cursor: pointer;
                        font-size: 16px;
                    }
                    button:hover {
                        background: #45a049;
                    }
                    .result-container {
                        margin-top: 30px;
                        border-top: 1px solid #ddd;
                        padding-top: 20px;
                    }
                    .result-card {
                        background: #f9f9f9;
                        border-radius: 8px;
                        padding: 20px;
                        margin-bottom: 20px;
                        border-left: 4px solid #4CAF50;
                    }
                    .result-card.error {
                        border-left-color: #f44336;
                    }
                    .metrics {
                        background: #e8f4fc;
                        padding: 15px;
                        border-radius: 8px;
                        margin-bottom: 20px;
                    }
                    table {
                        width: 100%;
                        border-collapse: collapse;
                        margin: 10px 0;
                    }
                    th, td {
                        padding: 12px;
                        text-align: left;
                        border-bottom: 1px solid #ddd;
                    }
                    th {
                        background-color: #f2f2f2;
                    }
                    .followup {
                        background: #fff3cd;
                        padding: 10px;
                        border-radius: 5px;
                        margin: 10px 0;
                    }
                    .highlight {
                        background-color: #ffffcc;
                        padding: 2px 4px;
                        border-radius: 3px;
                    }
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>✈️ Aviation Safety Query Dashboard</h1>

                    <div class="query-form">
                        <input type="text" id="queryInput" placeholder="Ask about aviation safety data, e.g., 'Show Boeing 737 accidents'">
                        <button onclick="executeQuery()">Search</button>
                        <button onclick="clearResults()" style="background: #f44336;">Clear</button>
                    </div>

                    <div class="metrics">
                        <h3>System Status</h3>
                        <div id="systemMetrics">Loading...</div>
                    </div>

                    <div id="loading" style="display: none; text-align: center;">
                        <p>Processing your query...</p>
                    </div>

                    <div id="results"></div>

                    <div style="margin-top: 40px; font-size: 12px; color: #666;">
                        <h4>Example Queries:</h4>
                        <div style="display: flex; flex-wrap: wrap; gap: 10px;">
                            <span class="highlight" onclick="setQuery(this)">Show Boeing 737 accidents</span>
                            <span class="highlight" onclick="setQuery(this)">Airbus A320 fatal incidents</span>
                            <span class="highlight" onclick="setQuery(this)">United Airlines safety record</span>
                            <span class="highlight" onclick="setQuery(this)">Accidents at LAX airport</span>
                            <span class="highlight" onclick="setQuery(this)">Most common injury types</span>
                        </div>
                    </div>
                </div>

                <script>
                    function setQuery(element) {
                        document.getElementById('queryInput').value = element.textContent;
                    }

                    function executeQuery() {
                        const query = document.getElementById('queryInput').value;
                        if (!query) return;

                        document.getElementById('loading').style.display = 'block';
                        document.getElementById('results').innerHTML = '';

                        fetch('/query', {
                            method: 'POST',
                            headers: {'Content-Type': 'application/json'},
                            body: JSON.stringify({question: query})
                        })
                        .then(response => response.json())
                        .then(data => {
                            console.log("Data received from Flask:", data);
                            document.getElementById('loading').style.display = 'none';
                            displayResults(data);
                            updateMetrics();
                        })
                        .catch(error => {
                            document.getElementById('loading').style.display = 'none';
                            document.getElementById('results').innerHTML = `
                                <div class="result-card error">
                                    <h3>Error</h3>
                                    <p>${error.message}</p>
                                </div>
                            `;
                        });
                    }

                    function displayResults(data) {
                        const resultsDiv = document.getElementById('results');

                        if (data.success) {
                            let html = `
                                <div class="result-card">
                                    <h3>${data.response_type || 'Results'}</h3>
                                    <p><strong>Summary:</strong> ${data.summary}</p>
                                    <p><strong>Execution Time:</strong> ${data.execution_metrics.time_seconds}s
                                    (${data.execution_metrics.cached ? 'cached' : 'fresh query'})</p>
                            `;

                            // Display insights
                            if (data.insights && data.insights.length > 0) {
                                html += `<div style="background: #e8f5e9; padding: 10px; border-radius: 5px; margin: 10px 0;">
                                    <strong>Insights:</strong><ul>`;
                                data.insights.forEach(insight => {
                                    html += `<li>${insight}</li>`;
                                });
                                html += `</ul></div>`;
                            }

                            // Display results table
                            if (data.results && data.results.length > 0) {
                                const columns = Object.keys(data.results[0]);
                                html += `<table><thead><tr>`;
                                columns.forEach(col => {
                                    html += `<th>${col.replace('_', ' ')}</th>`;
                                });
                                html += `</tr></thead><tbody>`;

                                data.results.slice(0, 10).forEach(row => {
                                    html += `<tr>`;
                                    columns.forEach(col => {
                                        html += `<td>${row[col] || 'N/A'}</td>`;
                                    });
                                    html += `</tr>`;
                                });
                                html += `</tbody></table>`;

                                if (data.results.length > 10) {
                                    html += `<p>Showing 10 of ${data.results.length} results</p>`;
                                }
                            }

                            // Display follow-up questions
                            if (data.follow_up_questions && data.follow_up_questions.length > 0) {
                                html += `<div class="followup">
                                    <strong>Suggested Follow-ups:</strong><br>`;
                                data.follow_up_questions.slice(0, 3).forEach(question => {
                                    html += `<span class="highlight" onclick="setQuery(this)" style="cursor: pointer; margin: 5px; display: inline-block;">${question}</span>`;
                                });
                                html += `</div>`;
                            }

                            // Display Cypher query
                            html += `<details style="margin-top: 20px;">
                                <summary><strong>Cypher Query</strong></summary>
                                <pre style="background: #f4f4f4; padding: 10px; border-radius: 5px; overflow-x: auto;">${data.raw_cypher}</pre>
                            </details>`;

                            html += `</div>`;
                            resultsDiv.innerHTML = html;
                        } else {
                            resultsDiv.innerHTML = `
                                <div class="result-card error">
                                    <h3>Query Failed</h3>
                                    <p>${data.error}</p>
                                    <strong>Suggestions:</strong>
                                    <ul>${(data.suggestions || []).map(s => `<li>${s}</li>`).join('')}</ul>
                                </div>
                            `;
                        }
                    }

                    function updateMetrics() {
                        fetch('/metrics')
                            .then(response => response.json())
                            .then(data => {
                                const metricsDiv = document.getElementById('systemMetrics');
                                metricsDiv.innerHTML = `
                                    <p><strong>Total Queries:</strong> ${data.performance.total_queries_processed}</p>
                                    <p><strong>Success Rate:</strong> ${(data.performance.success_rate * 100).toFixed(1)}%</p>
                                    <p><strong>Avg Time:</strong> ${data.performance.average_execution_time?.toFixed(2) || '0'}s</p>
                                    <p><strong>Cache Hit Rate:</strong> ${(data.performance.cache_hit_rate * 100).toFixed(1)}%</p>
                                `;
                            });
                    }

                    function clearResults() {
                        document.getElementById('results').innerHTML = '';
                        document.getElementById('queryInput').value = '';
                    }

                    // Load metrics on page load
                    document.addEventListener('DOMContentLoaded', updateMetrics);

                    // Allow Enter key to submit query
                    document.getElementById('queryInput').addEventListener('keypress', function(e) {
                        if (e.key === 'Enter') {
                            executeQuery();
                        }
                    });
                </script>
            </body>
            <center>
                <footer>
                    <p><small>&copy; 2025 Embry-Riddle Aeronautical University (ERAU). All rights reserved.</small></p>
                    <p><small><i>This dashboard is powered by a Knowledge Graph-based RAG system. Please note that system-generated content may be inaccurate.</i></small></p>

                </footer>
            </center>
            </html>
            ''')

        @self.app.route('/query', methods=['POST'])
        def query():
            data = request.json
            result = self.query_engine.query(data.get('question', ''))
            sanitized_result = sanitize_for_json(result)
            print("\nSanitized Result: ", sanitized_result, "\n")
            return jsonify(sanitized_result)

        @self.app.route('/metrics', methods=['GET'])
        def metrics():
            return jsonify(self.query_engine.get_system_metrics())

        def sanitize_for_json(obj):
            if isinstance(obj, list):
                return [sanitize_for_json(i) for i in obj]
            elif isinstance(obj, dict):
                return {k: sanitize_for_json(v) for k, v in obj.items()}
            elif isinstance(obj, float) and math.isnan(obj):
                return None  # Becomes 'null' in your browser
            return obj

api = AviationQueryAPI(query_engine)

def run_app():
    # Use threaded=True to handle multiple browser requests
    api.app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

# Start Flask in a background thread
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001


## Step 3: Exposing Local Development Server to Public
Note that this will expose your Colab based dev server to internet by giving it a temporary, shareable URL. The command below will give you the public IP address to be used.

In [ ]:
# This creates a public link for your Flask app
!npx localtunnel --port 5000

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://fluffy-pants-think.loca.lt


INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 16:14:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 16:14:24] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 16:14:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Dec/2025 16:14:35] "POST /query HTTP/1.1" 200 -


^C


In [ ]:
!curl https://loca.lt/mytunnelpassword

35.229.138.112